In [11]:
import pandas as pd

import pdfminer
import io
import os
from collections import OrderedDict 

from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [3]:
def download_pdfs():  #Download all pdfs in your local directory
    url = 'https://www.google.com/covid19/mobility/'
    response = requests.get(url)

    soup = BeautifulSoup(response.text, "html.parser")
    soup.findAll('a', {"class": "download-link"})
    new_files = False
    
    if not os.path.exists('data'):
        os.makedirs('data')   #os. makedirs() method in Python is used to create a directory recursively.

    for one_a_tag in soup.findAll('a', {"class": "download-link"}):
        link = one_a_tag['href']
        file_name = link[link.find('mobility')+len('mobility')+1:]
        path = 'data/' + file_name
        if not os.path.isfile(path):
            new_files = True
            urllib.request.urlretrieve(link, path)
            print(file_name)
            time.sleep(1)
    if not new_files:
        print('No updates')

In [4]:
def extract_text_from_pdf(pdf_path): 
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

In [5]:
def parse_covid_report(text, regions=False, attributes=['Retail & recreation', 'Grocery & pharmacy',
                                        'Parks', 'Transit stations',
                                        'Workplaces', 'Residential']):
    data = OrderedDict()
    if not regions:
        for i in range(len(attributes)):
            index = text.find(attributes[i])+len(attributes[i])
            if text[index]!=' ':
                data[attributes[i]] = int(text[index:index+text[index:].find('%')])
            else:
                data[attributes[i]] = None
    
    else:
        data['Region']=['Total']
        for i in range(len(attributes)):
            index = text.find(attributes[i])+len(attributes[i])
            if text[index]!=' ':
                data[attributes[i]] = data.get(attributes[i],[])+[int(text[index:index+text[index:].find('%')])]
            else:
                data[attributes[i]] = data.get(attributes[i],[]) + [None]
        
        
        last_index = text.find(attributes[len(attributes)-1])
        
        while True:
            if text[last_index+1:].find(attributes[0])<0:
                break
            reg_ind = 1
            while True:
                m = text[last_index+1:].find(attributes[0])
                region = text[last_index+1+m-reg_ind:last_index+1+m]
                
                xoc = region.find('\x0c')
                baseline = region.find('baseline')
                dat = region.find('date')
                
                if xoc>=0:
                    region = region[xoc+1:]
                    break
                if baseline>=0:
                    region = region[8:]
                    break
                if dat>=0:
                    region = region[4:]
                    break    
                
                reg_ind+=1
            
            data['Region'] += [region]
            
            text = text[last_index+text[last_index+1:].find(attributes[0]):]

            for i in range(len(attributes)):
                
                # masterpiece trick
                if attributes[i]!='Workplaces':
                    index = text.find(attributes[i])+len(attributes[i])
                else:
                    index = text.find('Workplace')+len('Workplace')
                
                if text[index:index+2]!=' N':
                    if text[index]!=' ':
                        data[attributes[i]] += [int(text[index:index+text[index:].find('%')])]
                    else:
                        data[attributes[i]] += [int(text[index+1:index+text[index:].find('%')])]
                else:
                    data[attributes[i]] += [None]

            last_index = text.find(attributes[len(attributes)-1])        
    
    return data

In [6]:
def build_excel_covid_report_total(directory):
    
    data = OrderedDict() 
    
    if os.path.isfile('codes.csv'):
        codes = pd.read_csv('codes.csv', sep=';',index_col=0,keep_default_na=False)
    else:
        codes = None
   
    for filename in os.listdir(directory):
        filename_list = filename.split('_')
        if len(filename_list)==5: #and filename not in files_source:
            text = extract_text_from_pdf(os.path.join(directory, filename))
            if codes is not None:
                country_name = codes.loc[filename_list[1],'Country'] if filename_list[1] in codes.index else filename_list[1]
            else:
                country_name = filename_list[1]
            data['Country'] = data.get('Country',[]) + [country_name]
            data['Date'] = data.get('Date',[]) + [filename_list[0]]
            parsed = parse_covid_report(text)
            for k,v in parsed.items():
                data[k] = data.get(k,[]) + [v]   
    return data

In [7]:
download_pdfs()

2020-03-29_AF_Mobility_Report_en.pdf
2020-03-29_AO_Mobility_Report_en.pdf
2020-03-29_AG_Mobility_Report_en.pdf
2020-03-29_AR_Mobility_Report_en.pdf
2020-03-29_AW_Mobility_Report_en.pdf
2020-03-29_AU_Mobility_Report_en.pdf
2020-03-29_AT_Mobility_Report_en.pdf
2020-03-29_BH_Mobility_Report_en.pdf
2020-03-29_BD_Mobility_Report_en.pdf
2020-03-29_BB_Mobility_Report_en.pdf
2020-03-29_BY_Mobility_Report_en.pdf
2020-03-29_BE_Mobility_Report_en.pdf
2020-03-29_BZ_Mobility_Report_en.pdf
2020-03-29_BJ_Mobility_Report_en.pdf
2020-03-29_BO_Mobility_Report_en.pdf
2020-03-29_BA_Mobility_Report_en.pdf
2020-03-29_BW_Mobility_Report_en.pdf
2020-03-29_BR_Mobility_Report_en.pdf
2020-03-29_BG_Mobility_Report_en.pdf
2020-03-29_BF_Mobility_Report_en.pdf
2020-03-29_KH_Mobility_Report_en.pdf
2020-03-29_CM_Mobility_Report_en.pdf
2020-03-29_CA_Mobility_Report_en.pdf
2020-03-29_CV_Mobility_Report_en.pdf
2020-03-29_CL_Mobility_Report_en.pdf
2020-03-29_CO_Mobility_Report_en.pdf
2020-03-29_CR_Mobility_Report_en.pdf
2

In [8]:
# build total world report
total = pd.DataFrame(data = build_excel_covid_report_total('data'))
total.to_excel('mobility_total_world.xlsx', sheet_name = 'Total data', index = False)

In [9]:
def build_excel_covid_report_detailed(directory = 'data',destination ='mobility_report' ,
                                      report_type = 'regions', file_format = 'excel', tabs = 'multiple'):
    """
    data - path of downloaded pdfs
    destination - destination file of report (without format)
    report_type: 'regions', 'US_states'
    format: 'excel', 'csv'
    tabs: ''multiple', 'single'
    """
    
    if os.path.isfile('codes.csv'):
        codes = pd.read_csv('codes.csv', sep=';',index_col=0,keep_default_na=False)
    else:
        codes = None
        
    if report_type != 'regions' and report_type != 'US_states':
        raise NameError("Wrong report_type. Available options: 'regions', 'US_states'")
    
    all_data = OrderedDict()
    reg_list=[]
    
    for filename in os.listdir(directory):
        filename_list = filename.split('_')    
        if report_type == 'regions':
            if len(filename_list)==5:
                text = extract_text_from_pdf(os.path.join(directory, filename))
                if codes is not None:
                    country_name = codes.loc[filename_list[1],'Country'] if filename_list[1] in codes.index else filename_list[1]
                else:
                    country_name = filename_list[1]
            else: continue
                    
        elif report_type == 'US_states':
            if len(filename_list)>=6:
                text = extract_text_from_pdf(os.path.join(directory, filename))
                country_name = filename_list[2]
                if len(filename_list)>=7: country_name+=' '+filename_list[3]
                if len(filename_list)>=8: country_name+=' '+filename_list[4]
            else: continue
            
        parsed = parse_covid_report(text, regions=True)
        reg_name = 'Country' if report_type == 'regions' else 'State'
        reg_list.append(country_name)
        all_data['Date'] = all_data.get('Date',[]) + [filename_list[0] for i in range(len(parsed['Region']))]
        all_data[reg_name] = all_data.get(reg_name,[])+ [country_name for i in range(len(parsed['Region']))]
        for k,v in parsed.items():
            all_data[k] = all_data.get(k,[]) + v
    
    df = pd.DataFrame(data=all_data)
    if file_format == 'excel':
        if tabs == 'single':
            writer = pd.ExcelWriter(destination + '.xlsx', engine = 'xlsxwriter')
            df.to_excel(writer,sheet_name='Regions', index=False)
            writer.save()
            writer.close()
        else:
            writer = pd.ExcelWriter(destination + '.xlsx', engine = 'xlsxwriter')
            for r in reg_list:
                df_reg = df.loc[df[reg_name] == r]
                df_reg = df_reg.drop(reg_name,1)
                df_reg.to_excel(writer,sheet_name=r, index=False)
            writer.save()
            writer.close()
    else:
        df.to_csv(destination + '.csv', index = False)

In [10]:
build_excel_covid_report_detailed(directory = 'data', destination = 'mobility_US_states',
                                  report_type = 'US_states', file_format = 'csv')

In [14]:
US = pd.read_csv('mobility_US_states.csv')
US

,Date,State,Region,Retail & recreation,Grocery & pharmacy,Parks,Transit stations,Workplaces,Residential
0,2020-03-29,Alabama,Total,-41.0,-13.0,19.0,-30.0,-32.0,9.0
1,2020-03-29,Alabama,Autauga County,-42.0,-8.0,-14.0,NaN,-35.0,16.0
2,2020-03-29,Alabama,Baldwin County,-47.0,-19.0,-27.0,-24.0,-32.0,9.0
3,2020-03-29,Alabama,Barbour County,-27.0,-20.0,NaN,NaN,-23.0,NaN
4,2020-03-29,Alabama,Bibb County,-25.0,-10.0,NaN,NaN,-32.0,NaN
...,...,...,...,...,...,...,...,...,...
2855,2020-03-29,Wyoming,Sweetwater County,-33.0,-9.0,NaN,6.0,-22.0,15.0
2856,2020-03-29,Wyoming,Teton County,-83.0,-50.0,-57.0,-80.0,-70.0,NaN
2857,2020-03-29,Wyoming,Uinta County,-29.0,2.0,NaN,1.0,-18.0,NaN
2858,2020-03-29,Wyoming,Washakie County,-17.0,-7.0,NaN,NaN,-32.0,NaN


In [15]:
US = US.loc[US['Region'] == 'Total']

US.drop(US.columns[[2]], axis = 1, inplace = True)
US

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Date,State,Retail & recreation,Grocery & pharmacy,Parks,Transit stations,Workplaces,Residential
0,2020-03-29,Alabama,-41.0,-13.0,19.0,-30.0,-32.0,9.0
68,2020-03-29,Alaska,-48.0,-27.0,18.0,-55.0,-33.0,10.0
81,2020-03-29,Arizona,-40.0,-17.0,-7.0,-41.0,-33.0,10.0
97,2020-03-29,Arkansas,-29.0,-7.0,81.0,-22.0,-27.0,7.0
169,2020-03-29,California,-50.0,-24.0,-38.0,-54.0,-39.0,15.0
226,2020-03-29,Colorado,-51.0,-27.0,-12.0,-60.0,-40.0,13.0
276,2020-03-29,Connecticut,-56.0,-32.0,-52.0,-64.0,-38.0,15.0
285,2020-03-29,Delaware,-47.0,-28.0,-6.0,-57.0,-37.0,12.0
289,2020-03-29,District of Columbia,-64.0,-30.0,-41.0,-68.0,-47.0,14.0
290,2020-03-29,Florida,-50.0,-26.0,-48.0,-63.0,-41.0,13.0
